In [ ]:
import tensorflow as tf
import os
import keras
import numpy as np
import easygdf
import mne
import json
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, MaxPooling1D, Dropout, Flatten
from tensorflow.keras import layers, models
from tensorflow.keras.regularizers import l2
import get_data
from sklearn.model_selection import train_test_split
from colabcode import ColabCode

In [ ]:
def event_position():

    filenames = ["A01T.gdf", "A01E.gdf", "A02T.gdf", "A02E.gdf","A03T.gdf","A03E.gdf","A04T.gdf","A04E.gdf","A05T.gdf","A05E.gdf","A06T.gdf","A06E.gdf","A07T.gdf","A07E.gdf","A08T.gdf","A08E.gdf","A09T.gdf","A09E.gdf"]
    labels = ["A01T", "A01E", "A02T", "A02E","A03T","A03E","A04T","A04E","A05T","A05E","A06T","A06E","A07T","A07E","A08T","A08E","A09T","A09E"]


    events_with_labels_arrays = {}

    for filename, label in zip(filenames, labels):

        filename='../datasets/BCICIV_2a_gdf/'+filename
        raw = mne.io.read_raw_gdf(filename)

        # Access event information from the raw object
        events, event_id = mne.events_from_annotations(raw)

        #the list have the dictionary that map each events' id  to events' labels
        events_id_labels = list(event_id.keys())

        #create array for gather togther events' type and positions
        temp= np.zeros_like(events)


        for i in range(len(events)):
            temp[i,0]=events[i,0]
            id=events[i,2]
            temp[i,1]=events_id_labels[id-1] # be in dalil ke dar list event ha ma id 10 drim vali araye events_id_labels k az tarigh key dastresi peyda krdim yedone adadash paiin tare

        events_with_labels_arrays[label] = temp

    
    # Get the sample indices of the events
    #event_positions_samples = events[:, 0]

    # Get the corresponding time points from raw
    #event_positions_times = raw.times[event_positions_samples]    

    return events_with_labels_arrays

In [ ]:
def save_data_in_array():

    filenames = ["A01T.gdf", "A01E.gdf", "A02T.gdf", "A02E.gdf","A03T.gdf","A03E.gdf","A04T.gdf","A04E.gdf","A05T.gdf","A05E.gdf","A06T.gdf","A06E.gdf","A07T.gdf","A07E.gdf","A08T.gdf","A08E.gdf","A09T.gdf","A09E.gdf"]
    labels = ["A01T", "A01E", "A02T", "A02E","A03T","A03E","A04T","A04E","A05T","A05E","A06T","A06E","A07T","A07E","A08T","A08E","A09T","A09E"]


    # Create a dictionary to store data arrays
    data_arrays = {}


    # Iterate over the filenames and labels
    for filename, label in zip(filenames, labels):
        # Read the raw data
        filename='../datasets/BCICIV_2a_gdf/'+filename
        raw = mne.io.read_raw_gdf(filename)
        
        # Convert to DataFrame
        df = raw.to_data_frame()
        
        # Store data array in the dictionary
        data_arrays[label] = df.values  # Store the numpy array representation of the DataFrame

    # # Access data array for a specific label
    # example_label = "A01T"
    # example_data_array = data_arrays[example_label]    

    return data_arrays

In [ ]:
def create_training_data(data_arrays,events_with_labels_arrays):
    labels = ["A01T", "A02T", "A03T","A04T","A05T","A06T","A07T","A08T","A09T"]

    X_train=[]

    for label in labels:

        temp_data=data_arrays[label]
        temp_events=events_with_labels_arrays[label]
        for i in range(len(temp_events)):
        
            if temp_events[i,1]==769 or temp_events[i,1]==770 or temp_events[i,1]==771 or temp_events[i,1]==772 :

                ev=temp_events[i,1]
                position=temp_events[i,0]
                for j in range(position, position+1000):

                    temp=np.append(temp_data[j,1:23],ev)
                    X_train.append(temp)

In [ ]:
# Define the squeeze excitation block function
def squeeze_excitation_block(inputs, r):
    filters = inputs.shape[-1]
    se = layers.GlobalAveragePooling1D()(inputs)
    se = layers.Dense(filters // r, activation='relu')(se)
    se = layers.Dense(filters, activation='sigmoid')(se)
    se = layers.Reshape((1, filters))(se)
    return layers.multiply([inputs, se])

# Set random seed for reproducibility
tf.random.set_seed(42)

# Define input shape
input_shape = (22,1)

# Define input layers for each branch
input_1 = tf.keras.Input(shape=input_shape)
input_2 = tf.keras.Input(shape=input_shape)
input_3 = tf.keras.Input(shape=input_shape)

# First Branch
x1 = layers.Conv1D(32, 20, strides=2, padding='same', kernel_regularizer=l2(0.01))(input_1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.ELU()(x1)
x1 = squeeze_excitation_block(x1, r=8)
x1 = layers.MaxPooling1D()(x1)

x1 = layers.Conv1D(32, 9, strides=2, padding='same', kernel_regularizer=l2(0.01))(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.ELU()(x1)
x1 = squeeze_excitation_block(x1, r=8)
x1 = layers.MaxPooling1D()(x1)

x1 = layers.Conv1D(32, 5, strides=1, padding='same', kernel_regularizer=l2(0.01))(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.ELU()(x1)

x1 = layers.Flatten()(x1)

# Second Branch
x2 = layers.Conv1D(32, 13, strides=2, padding='same', kernel_regularizer=l2(0.01))(input_2)
x2 = layers.BatchNormalization()(x2)
x2 = layers.ELU()(x2)
x2 = squeeze_excitation_block(x2, r=8)

x2 = layers.Conv1D(32, 7, strides=2, padding='same', kernel_regularizer=l2(0.01))(x2)
x2 = layers.BatchNormalization()(x2)
x2 = layers.ELU()(x2)
x2 = squeeze_excitation_block(x2, r=8)

x2 = layers.Conv1D(32, 5, strides=1, padding='same', kernel_regularizer=l2(0.01))(x2)
x2 = layers.BatchNormalization()(x2)
x2 = layers.ELU()(x2)

x2 = layers.Flatten()(x2)

# Third Branch
x3 = layers.Conv1D(32, 15, strides=2, padding='same', kernel_regularizer=l2(0.01))(input_3)
x3 = layers.BatchNormalization()(x3)
x3 = layers.ELU()(x3)
x3 = squeeze_excitation_block(x3, r=8)

x3 = layers.Conv1D(32, 10, strides=2, padding='same', kernel_regularizer=l2(0.01))(x3)
x3 = layers.BatchNormalization()(x3)
x3 = layers.ELU()(x3)
x3 = squeeze_excitation_block(x3, r=8)

x3 = layers.Conv1D(32, 5, strides=1, padding='same', kernel_regularizer=l2(0.01))(x3)
x3 = layers.BatchNormalization()(x3)
x3 = layers.ELU()(x3)

x3 = layers.Flatten()(x3)

# Concatenate the outputs of three branches
merged = layers.concatenate([x1, x2, x3])

# Dropout layer
x = layers.Dropout(0.5)(merged)

# # Flatten layer
# x = layers.Flatten()(x)

# # Dense layer
# x = layers.Dense(64, activation='relu')(x)

# Output layer
output = layers.Dense(4, activation='softmax')(x)

# Create model
model = Model(inputs=[input_1, input_2, input_3], outputs=output)

# Compile model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

In [ ]:
events_with_labels_arrays=get_data.event_position()

data_arrays=get_data.save_data_in_array()

In [ ]:
#add label for each row as last column
train_data=get_data.create_training_data(data_arrays,events_with_labels_arrays)

In [ ]:
#convert train_data to np array
train_data = np.array(train_data)

#Separate features (X) and labels (y)
X = train_data[:, :-1]  # Extract all rows and all columns except the last one
Y = train_data[:, -1]   # Extract all rows and only the last column

#reshape Y to 2d array
Y = Y.reshape(-1, 1)

In [ ]:
for i in range(len(Y)):
    if Y[i]==769:
        Y[i]=0
    elif Y[i]==770:
        Y[i]=1
    elif Y[i]==771:
        Y[i]=2    
    elif Y[i]==772:
        Y[i]=3    

In [ ]:
x_train, x_, y_train, y_ =train_test_split(X, Y, test_size=0.20, random_state=1)

# x_cv, x_test, y_cv, y_test =train_test_split(x_, y_, test_size=0.50, random_state=1)

# del x_ ,y_

In [ ]:
# Generate shuffled indices
indices = np.arange(len(x_train))
np.random.shuffle(indices)

# Shuffle x_train and y_train using the shuffled indices
x_train_shuffled = x_train[indices]
y_train_shuffled = y_train[indices]

# Now x_train_shuffled and y_train_shuffled are shuffled while maintaining correspondence

In [ ]:
# Train the model
history=model.fit([x_train_shuffled, x_train_shuffled, x_train_shuffled], y_train_shuffled, epochs=400, batch_size=64, validation_split=0.2)

# Plot the training loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show() 